In [1]:
import psycopg2
import pandas as pd
import conf
from src.query.utils import df_query

In [2]:
dbc = psycopg2.connect(user="jupytersvc",
                        password="password",
                        host="172.21.0.2",
                        port="5432",
                        database="netzerodb")

In [3]:
cursor = dbc.cursor()

In [5]:
df_query(cursor,"SELECT table_name FROM information_schema.tables WHERE table_schema='public' AND table_type='BASE TABLE';")

,table_name
0,dim_network
1,dim_batch
2,dim_region
3,fact_power
4,dim_fueltech
5,fact_price
6,fact_x_cap
7,fact_carbonfactor


In [6]:
df_query(cursor,"select * from fact_carbonfactor")

,network_id,region_id,fueltech_id,carbon_factor
0,1,1,1,0.75
1,1,1,2,0.0
2,1,1,3,0.51
3,1,1,5,0.0
4,1,1,7,0.0
5,1,1,8,0.91
6,1,2,1,0.9845777040000001
7,1,2,3,0.4737075
8,1,2,5,0.0
9,1,2,7,0.0


In [5]:
co2_parameters = pd.read_csv("data/co2_parameters.csv",index_col=1)
co2_parameters.index = co2_parameters.index.str.lower()
co2_parameters = co2_parameters.drop(columns=["Uid"])
co2_parameters = co2_parameters

In [6]:
co2_parameters

,NSW1,VIC1,QLD1,WA1,SA1,TAS1
Fueltech,,,,,,
gas_ocgt,0.75,0.75000,0.810,0.750,0.984578,0.66
gas_ccgt,0.51,NaN,0.510,0.510,0.473708,0.70
distillate,NaN,NaN,NaN,NaN,NaN,NaN
battery_discharging,NaN,0.00000,NaN,NaN,0.000000,NaN
gas_steam,NaN,0.68115,NaN,NaN,0.681150,NaN
hydro,0.00,0.00000,0.000,NaN,NaN,0.00
battery_charging,NaN,0.50000,NaN,NaN,0.500000,NaN
coal_black,0.91,NaN,0.880,0.820,NaN,NaN
gas_wcmg,NaN,NaN,0.490,NaN,NaN,NaN


In [8]:
Fact_CarbonFactor = df_query(cursor,
"select \
    dim_network.id as network_id,\
    dim_region.id as region_id,\
    dim_fueltech.id as fueltech_id,\
    dim_region.short_name,\
    dim_fueltech.long_name \
from dim_network, dim_region, dim_fueltech"
)
Fact_CarbonFactor["carbon_factor"] = 0
for i in range(Fact_CarbonFactor.shape[0]):
    region, fueltech = Fact_CarbonFactor.iloc[i][["short_name","long_name"]]
    Fact_CarbonFactor.loc[i,"carbon_factor"] =  co2_parameters.loc[fueltech,region]
Fact_CarbonFactor = Fact_CarbonFactor[["network_id","region_id","fueltech_id","carbon_factor"]]

In [9]:
data = Fact_CarbonFactor[Fact_CarbonFactor["carbon_factor"].notnull()]
for i in range(data.shape[0]):
    network_id,region_id,fueltech_id,carbon_factor = data.iloc[i][["network_id","region_id","fueltech_id","carbon_factor"]]
    cursor.execute(
    f"INSERT INTO Fact_CarbonFactor(network_id,region_id,fueltech_id,carbon_factor) VALUES ({network_id},{region_id},{fueltech_id},{carbon_factor})"
                )

In [10]:
df_query(cursor,"select * from fact_carbonfactor")

,network_id,region_id,fueltech_id,carbon_factor
0,1,1,1,0.75
1,1,1,2,0.0
2,1,1,3,0.51
3,1,1,5,0.0
4,1,1,7,0.0
5,1,1,8,0.91
6,1,2,1,0.9845777040000001
7,1,2,3,0.4737075
8,1,2,5,0.0
9,1,2,7,0.0


In [11]:
dbc.commit()